## Import Packages

In [1]:
import os
import sys
import sqlite3

import spotipy
import spotipy.util as util
from spotipy.client import SpotifyException

from collections import Counter
from gensim.utils import tokenize

import time
import tqdm
import json
import ndjson
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests

from creds import *

## Get Data

In [2]:
playlists = json.load(open("data/playlists.json"))

## Define Get Tracks

In [3]:
def track_yielder(session, playlist):
    res = session.user_playlist_tracks(playlist['owner'], playlist['id'],
              fields = 'items(track(id, name, artists(name, id), duration_ms)),next')
    while res:
        for track in res['items']:
            if track['track']:
                yield track['track']
        tries = 3
        while tries > 0:
            try:
                res = session.next(res)
                if not res or not res.get('items'):
                    return
                tries = 0
            except SpotifyException as e:
                if 400 <= e.http_status <= 499:
                    raise StopIteration
                tries -= 1
                time.sleep(1)
                if tries == 0:
                    raise e

## Get Tracks

In [4]:
# Start Session
session = spotipy.Spotify(auth_manager = spotipy.SpotifyOAuth(
    client_id = client_id,
    client_secret = client_secret,
    scope = scope,
    username = user_id,
    redirect_uri = redirect_uri
))

In [6]:
if os.path.isfile('data/songs.db'):
    os.remove('data/songs.db')
conn = sqlite3.connect('data/songs.db')
c = conn.cursor()
c.execute('CREATE TABLE songs (id text primary key, name text, artist text)')
c.execute('CREATE INDEX name_idx on songs(name)')
count = 0
tracks_seen = set()
with open('data/playlists.ndjson', 'w') as fout_playlists:
    with open('data/songs_ids.txt', 'w') as fout_song_ids:
        for playlist in tqdm.tqdm_notebook(playlists.values()):
            try:
                fout_playlists.write(json.dumps(playlist) + '\n')
                track_ids = []
                for track in track_yielder(session, playlist):
                    track_id = track['id']
                    if not track_id:
                        continue
                    if not track_id in tracks_seen:
                        c.execute("INSERT INTO songs VALUES (?, ?, ?)", 
                                  (track['id'], track['name'], track['artists'][0]['name']))
                    track_ids.append(track_id)
                    tracks_seen.add(track_id)
                fout_song_ids.write(' '.join(track_ids) + '\n')
                conn.commit()
            except:
                session = spotipy.Spotify(auth_manager = spotipy.SpotifyOAuth(
                    client_id = client_id,
                    client_secret = client_secret,
                    scope = scope,
                    username = user_id,
                    redirect_uri = redirect_uri
                ))
                count += 1
                print(f"Playlist Failures: {count}")
                print(playlist)
conn.commit()

/Users/grantwilson/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


Playlist Failures: 1
{'owner': 'spotify', 'name': 'Cosa Fai a Capodanno?', 'id': '37i9dQZF1DX3n3r7GdzC0V'}
Playlist Failures: 2
{'owner': 'elmirabastos', 'name': 'its nice to have a friend', 'id': '4qnBcmsIozsafpdBZJPuYj'}
Playlist Failures: 3
{'owner': 'alequintana16', 'name': 'Piensa en mi, Llora por mi, Llamame a mi💃🏻', 'id': '6JcYnCxMPHrJ24BDmh8kBk'}
Playlist Failures: 4
{'owner': 'somebodyonearth', 'name': 'Christmas Lofi', 'id': '4pT9fDRHiSwVCU8nAaWt5V'}
Playlist Failures: 5
{'owner': 'py2rw3xospafv7fabhwxrgzxx', 'name': 'Christmas Music: Instrumentals', 'id': '1oHZbeYdaN0mf0Eec1gcAk'}
Playlist Failures: 6
{'owner': 'spotify', 'name': 'An Amapiano Christmas', 'id': '37i9dQZF1DX1sluhyhW7aP'}
Playlist Failures: 7
{'owner': '1258839312', 'name': 'Kids Hip Hop Christmas', 'id': '01jWtTjddHccTIxPTsyros'}
Playlist Failures: 8
{'owner': '1252064515', 'name': 'ITS CHRISTMAS', 'id': '2SFYP02F0lFn4dTHlzXsaR'}
Playlist Failures: 9
{'owner': '1160099727', 'name': 'Old songs🔥', 'id': '6WWqCw0

Playlist Failures: 72
{'owner': 'loz7002', 'name': 'rnb 00s', 'id': '2rvK4yAUPncGPRA2hgjCJE'}
Playlist Failures: 73
{'owner': '2a8y3o1ltc4o9sbulr4iomf3k', 'name': 'rnb', 'id': '0Fp51InLDJ2xYJNxHRVzOy'}
Playlist Failures: 74
{'owner': 'spotify', 'name': 'Ev Partisi', 'id': '37i9dQZF1DX555axolvGg3'}
Playlist Failures: 75
{'owner': '31ichxlo7peexkbd4yy6wol7srae', 'name': "Live From Daryl's House", 'id': '6S0BMYToT5EovOvA3Wac8b'}
Playlist Failures: 76
{'owner': '31kjnqudwciefmp72bfuhnyxuxre', 'name': 'Deep', 'id': '5Enq2K9qrVlvogGE8SHWQ0'}
Playlist Failures: 77
{'owner': 'py2rw3xospafv7fabhwxrgzxx', 'name': '8 Hours: Deep Sleep', 'id': '6ItX7QUbiCSrZh07u8nVuM'}
Playlist Failures: 78
{'owner': 'daviddeanburkhart', 'name': 'Compact Cassette', 'id': '6lzG8MsMBoVEpulGS747M6'}
Playlist Failures: 79
{'owner': 'vub0l7wkstrkqgm0o2whrq0n7', 'name': 'Chill Vibes', 'id': '6AJu17bi9KwyVE95lIL4LH'}
Playlist Failures: 80
{'owner': 'py2rw3xospafv7fabhwxrgzxx', 'name': 'Christmas Jazz: Vintage Lounge', 'i

Playlist Failures: 142
{'owner': '1288637360', 'name': 'Reggaeton', 'id': '0suR2bG4f8nwzKsHht1Jsr'}
Playlist Failures: 143
{'owner': 'edgar_f_luv', 'name': 'HIGH SONGS', 'id': '06lyE84pjuHyiiN8Lwor65'}
Playlist Failures: 144
{'owner': '21xpqjftnqpman5d3jmc7icwi', 'name': 'high', 'id': '1ULwUVhWpZ0Ly1Bm20J7kv'}
Playlist Failures: 145
{'owner': '1277461273', 'name': 'Old but Gold', 'id': '5yCZN54qhOD4F48jhzWu3x'}
retrying ...1secs
Playlist Failures: 146
{'owner': '12133674014', 'name': "D' Gold 80'90", 'id': '6SEFdWZdGr13seyynhguix'}
Playlist Failures: 147
{'owner': '118248855', 'name': 'Shqip hits/oldies', 'id': '4ldpJQsD6O2fOYbLqEtoeV'}
Playlist Failures: 148
{'owner': 'hen2n6rxk7a2zw404br2co20z', 'name': 'Rock classico oldies', 'id': '2GTWGLxNW8Qy5lhXkXoOqb'}
Playlist Failures: 149
{'owner': 'fs67g12s2if1btmcd046xelbz', 'name': 'oldies but goodies', 'id': '65gBzMMsJwo8Xj43XeLDmQ'}
Playlist Failures: 150
{'owner': 'nadiya2001', 'name': 'The Goodies', 'id': '1BvouYpUMpCzPkVraq6qM7'}
Pla

Playlist Failures: 212
{'owner': 'samanmedisl2004', 'name': 'getting ready ', 'id': '0L6ByQ7ndUUYu1aMWSujYq'}
Playlist Failures: 213
{'owner': 'gabbagazelle', 'name': 'EDC ready', 'id': '26ubf79iy1H36qiiTD8U8u'}
Playlist Failures: 214
{'owner': '216l6crn4k4o2gzurdgqdopiq', 'name': 'Get ready bitch ', 'id': '2DM2CmFylp1nO56In2cOSa'}
Playlist Failures: 215
{'owner': 'tqhqu105us69zvhpp6bk0rho7', 'name': 'get ready', 'id': '0f242jTpGuWKkXy0hnT82z'}
Playlist Failures: 216
{'owner': 'f60v595wo225evio0mg19uvp3', 'name': 'songs to help with a crush', 'id': '1rf4f7C9dPrjtPEh7EbmXy'}
Playlist Failures: 217
{'owner': 'cw2ka5um58jiglhhucgt6dhz0', 'name': 'getting over him✌️✌️✌️🙋\u200d♀️', 'id': '1e4zgxmVU7SNQRCTFuegtw'}
Playlist Failures: 218
{'owner': 'i8kw5w68jn6k07dwb98efqrz5', 'name': '#BeHappier in Love', 'id': '3eNjIlOI47ixLzffQV5Y45'}
Playlist Failures: 219
{'owner': '1185937179', 'name': 'SSX3', 'id': '2Pmy6F2FHOFKJOmvFtffpd'}
Playlist Failures: 220
{'owner': '22bdejycrmzpbwucwcwynqixq', '

...

Playlist Failures: 21992
{'owner': '12152686534', 'name': 'Resenha da Blakk', 'id': '6F4jkEF2YfVROJX48V9v6d'}
Playlist Failures: 21993
{'owner': '11131477160', 'name': 'Jon Skywalker playlist 2018/2019', 'id': '2rEQYyJRVBk5Xneftl8nMN'}
Playlist Failures: 21994
{'owner': '12175582402', 'name': 'Favoritas da Nah', 'id': '3HsDnYL1M0nelUh4J91boz'}
Playlist Failures: 21995
{'owner': 'b6575ef5junpxvrjpqe12yz5d', 'name': 'Da uma cavalgada', 'id': '58ORch2wdoNlvEcVVr1hDN'}
Playlist Failures: 21996
{'owner': '1170752292', 'name': 'Canzoni Disney Italiane', 'id': '7M3BD4Xxxqes9Lx7Kr2dd3'}
Playlist Failures: 21997
{'owner': '12143007823', 'name': 'Claros Breus', 'id': '3W3WSIKqIrpiNAjPkkIRpl'}
Playlist Failures: 21998
{'owner': '22pfppiuyeedbivcvotoh5gzi', 'name': 'BAILE DA CAPANEMA', 'id': '2VjwxrSPUOGGJQ88Yu3uSF'}
Playlist Failures: 21999
{'owner': '12153540012', 'name': 'Baile da Favorita 🔥', 'id': '1jwUqqyZripOvnu7BFhPRF'}
Playlist Failures: 22000
{'owner': 'iepferraz', 'name': 'Igreja Evangé

retrying ...1secs
Playlist Failures: 22059
{'owner': '2vq85dl3glos5u7435v2qkbbl', 'name': 'Most listend', 'id': '4ukIS9g0V7lRqqb0RH3nNi'}
Playlist Failures: 22060
{'owner': 'i8kw5w68jn6k07dwb98efqrz5', 'name': '#BeHappier this Christmas', 'id': '5yHx5a5EiFCBuDxA2deeci'}
retrying ...1secs
Playlist Failures: 22061
{'owner': '8z4e4v1mj1mzylq2wusz6ls9c', 'name': 'More listened to', 'id': '0BJ3oSrpFuGfdVHxeaR7pY'}
Playlist Failures: 22062
{'owner': 'uxnunafgvnav05yksq8bns57h', 'name': 'tik tok songs i’ve listened to too many times✨', 'id': '3gDuR1u4ITMRFAx4vBFbA3'}
Playlist Failures: 22063
{'owner': 'alasktree', 'name': 'nineteen', 'id': '5bYpGNfTTDXy3gVABTJoNm'}
Playlist Failures: 22064
{'owner': 'v1tt5fygj6fihm05btr1tupig', 'name': 'added recently', 'id': '2iVSwVdIdaONZc0GVTulv2'}
Playlist Failures: 22065
{'owner': 'rich_london', 'name': 'stuff ive heard recently', 'id': '0Tb9PFxZK9YZitf3kZ8Oju'}
Playlist Failures: 22066
{'owner': 'j8t31k11v8372spbesuqunj6c', 'name': 'recently added', 'id

Playlist Failures: 22125
{'owner': '21lxdcxaqhr4ytpjch4xixbnq', 'name': 'In meiner Zone', 'id': '5wwU7WIcgJkwRDf54PMflF'}
Playlist Failures: 22126
{'owner': 'mf6vfsdvrspd279wqent21tf5', 'name': 'a ZONE Workout Megaplaylist', 'id': '7b5zKqDxnJrWOxg5ulgbJr'}
Playlist Failures: 22127
{'owner': '9lu4vivpw6flxw7nr4p2ox6vp', 'name': 'food for heart', 'id': '1JXg5ccGnQ3dWuDqB4V7R8'}
Playlist Failures: 22128
{'owner': 'in1e1osf44s5u9hvw2mp0an2h', 'name': 'Soul Food', 'id': '12wyY6QPsYFH27OjaUtRRR'}
Playlist Failures: 22129
{'owner': '1220358069', 'name': 'My Brain', 'id': '3FbCWzboGVwoyFIxnPIRxR'}
Playlist Failures: 22130
{'owner': 'wp5p38o66xg2d29tushe2hsng', 'name': 'Brain', 'id': '5VgcbtoKQzky8Rx0cFe2rX'}
Playlist Failures: 22131
{'owner': 'jellieboat', 'name': 'Brain Fuzzies', 'id': '1EY0Yru15cZQgKHg2urygt'}
Playlist Failures: 22132
{'owner': '216f6zvjetyyyonj7vlcva3dy', 'name': 'Brain Bass', 'id': '68A06qGbCBYGFI2LqrJjgW'}
Playlist Failures: 22133
{'owner': 'megaartists', 'name': 'Tu tu n

Playlist Failures: 22191
{'owner': '123286315', 'name': 'PRAISE', 'id': '3YPZ5Bmj1pw8DT7h8EIqC6'}
Playlist Failures: 22192
{'owner': 'brianshort16', 'name': 'Praise', 'id': '0Z3NPbQj6ye5E3AJCRas0x'}
Playlist Failures: 22193
{'owner': '3vdgh35gt5dljw7crucciwwsk', 'name': 'praise', 'id': '7HYjATXTb2iNBNd2VMxjlX'}
Playlist Failures: 22194
{'owner': '1255361442', 'name': 'Praise & Worship', 'id': '0vVKRTrMazLld1qarK924B'}
Playlist Failures: 22195
{'owner': '12186119820', 'name': 'Praise', 'id': '5cN8yDMWHpDqiAC6sTaJ2P'}
Playlist Failures: 22196
{'owner': 'ana.lynn', 'name': 'praise & worship ', 'id': '4kV6eT6cQ7u2QOUSsArp2X'}
Playlist Failures: 22197
{'owner': 'emthegymnast5$', 'name': 'him✰', 'id': '4Bqqypr36QtTHklmVMTK7M'}
Playlist Failures: 22198
{'owner': 'merygemelier-55', 'name': 'dont know', 'id': '24fzZjsFc3sNDH2dGCnB6t'}
retrying ...1secs
Playlist Failures: 22199
{'owner': '21waz5tr4evqpqumvscbsd4xq', 'name': '🍇𝓭𝓾𝓶𝓫𝓪𝓼𝓼', 'id': '6PW1gIvBmi5mZYhoq3stU1'}
Playlist Failures: 22200
{'o

Playlist Failures: 22261
{'owner': 'gataextraordinaria', 'name': 'Una flamenquita ', 'id': '4FQWWkNmqVc6RjTayJcVCF'}
Playlist Failures: 22262
{'owner': 'alframso0511', 'name': 'Quien No Dice Una Mentira', 'id': '7aB2taHcLOK71lj8nitWFC'}
Playlist Failures: 22263
{'owner': 'sthefismartinez', 'name': 'Una mezcla importante', 'id': '2rEvoT2l8mVJlZdAuHBBId'}
Playlist Failures: 22264
{'owner': 'megaartists', 'name': 'Fuiste mi error favorito Eladio Carrion', 'id': '0IW0Dieb6T7DBS9DByD44F'}

